# APACHE SPARK EXERCISES

# DAG • Broadcast Joins • Transformations • Actions •Partitions

# DOMAIN

Ride-Hailing Platform Analytics (Uber / Ola-like)

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder\
.appName("DAG and Broadcast demo")\
.getOrCreate()

# DATASET 1 — RIDES (Large Fact Table)

In [1]:
rides_data = [
("R001","U001","Hyderabad",12.5,240,"Completed"),
("R002","U002","Delhi",8.2,180,"Completed"),
("R003","U003","Mumbai",15.0,300,"Cancelled"),
("R004","U004","Bangalore",5.5,120,"Completed"),
("R005","U005","Hyderabad",20.0,360,"Completed"),
("R006","U006","Delhi",25.0,420,"Completed"),
("R007","U007","Mumbai",7.5,150,"Completed"),
("R008","U008","Bangalore",18.0,330,"Completed"),
("R009","U009","Delhi",6.0,140,"Cancelled"),
("R010","U010","Hyderabad",10.0,200,"Completed")
]
rides_cols = [
"ride_id",
"user_id",
"city",
"distance_km",
"duration_seconds",
"status"
]

In [4]:
rides_df = spark.createDataFrame(rides_data, rides_cols)

# DATASET 2 — CITY SURGE MULTIPLIERS (Small Lookup)

In [5]:
surge_data = [
("Hyderabad",1.2),
("Delhi",1.5),
("Mumbai",1.8),
("Bangalore",1.3)
]
surge_cols = ["city","surge_multiplier"]
surge_df = spark.createDataFrame(surge_data, surge_cols)

# EXERCISE SET 1 — TRANSFORMATIONS vs ACTIONS

#Exercise 1.1

Create a transformation pipeline that:

Filters only Completed rides.
Selects ride_id , city , distance_km

Tasks:
Do not trigger any action
Explain whether Spark executed anything

In [7]:
completed_rides_df = rides_df.filter(rides_df.status == "Completed")
selected_rides_df = completed_rides_df.select("ride_id", "city", "distance_km")

# To verify the schema without triggering an action (for demonstration, but typically this would cause an action)
selected_rides_df.printSchema()

# No action is triggered, so Spark has only built the execution plan (DAG) but not executed it.

root
 |-- ride_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- distance_km: double (nullable = true)



# Exercise 1.2

Trigger a single action on the pipeline.

Tasks:

Identify which line caused execution
Explain why previous lines did not execute

In [8]:
selected_rides_df.show()

+-------+---------+-----------+
|ride_id|     city|distance_km|
+-------+---------+-----------+
|   R001|Hyderabad|       12.5|
|   R002|    Delhi|        8.2|
|   R004|Bangalore|        5.5|
|   R005|Hyderabad|       20.0|
|   R006|    Delhi|       25.0|
|   R007|   Mumbai|        7.5|
|   R008|Bangalore|       18.0|
|   R010|Hyderabad|       10.0|
+-------+---------+-----------+



In Spark, transformations like filter() and select() are lazily evaluated, meaning they only define the execution plan (DAG) and don't perform any computation until an action, such as .show(), .count(), or .collect(), is called. Therefore, the previous lines that defined completed_rides_df and selected_rides_df did not execute until show() was invoked.

# EXERCISE SET 2 — DAG & LINEAGE

# Exercise 2.1

Create a transformation chain with:

Multiple filters
A column selection

Tasks:
Run explain(True)

Identify:
Logical plan
Optimized logical plan
Physical plan

In [9]:
filtered_selected_rides_df = rides_df.filter("status == 'Completed'") \
                                   .filter("distance_km > 10") \
                                   .select("ride_id", "city", "distance_km", "status")

filtered_selected_rides_df.explain(True)

== Parsed Logical Plan ==
'Project ['ride_id, 'city, 'distance_km, 'status]
+- Filter (distance_km#3 > cast(10 as double))
   +- Filter (status#5 = Completed)
      +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Analyzed Logical Plan ==
ride_id: string, city: string, distance_km: double, status: string
Project [ride_id#0, city#2, distance_km#3, status#5]
+- Filter (distance_km#3 > cast(10 as double))
   +- Filter (status#5 = Completed)
      +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Optimized Logical Plan ==
Project [ride_id#0, city#2, distance_km#3, status#5]
+- Filter ((isnotnull(status#5) AND isnotnull(distance_km#3)) AND ((status#5 = Completed) AND (distance_km#3 > 10.0)))
   +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Physical Plan ==
*(1) Project [ride_id#0, city#2, distance_km#3, status#5]
+- *(1) Filter ((is

The explain(True) command successfully displayed the different phases of Spark's query optimization:

Parsed Logical Plan: This is the initial, unoptimized representation of your transformations as you wrote them. You can see two separate Filter operations and a Project (select) operation.

Analyzed Logical Plan: Spark resolves all column references and validates the query. In this case, it confirms the data types and ensures all columns exist.

Optimized Logical Plan: Spark's Catalyst optimizer applies various rules to improve efficiency. Notice how the two Filter operations (status == 'Completed' and distance_km > 10) have been combined into a single Filter operation with an AND condition. This is an example of predicate pushdown, where filters are applied as early as possible to reduce the amount of data processed.

Physical Plan: This is the concrete execution strategy that Spark will use on the cluster. It shows the actual operations Spark will perform, such as scanning the existing RDD, then applying the combined filter, and finally projecting the selected columns. The *(1) indicates a single stage for these operations.

# Exercise 2.2

Reorder transformations (filter after join vs before join).
Tasks:
Compare DAGs
Identify which plan is more efficient and why

In [10]:
# Join first, then filter
joined_then_filtered_df = rides_df.join(surge_df, on="city", how="inner") \
                                .filter("status == 'Completed'") \
                                .filter("distance_km > 10")

print("\n--- Plan for Join then Filter ---")
joined_then_filtered_df.explain(True)

# Filter first, then join
filtered_then_joined_df = rides_df.filter("status == 'Completed'") \
                                .filter("distance_km > 10") \
                                .join(surge_df, on="city", how="inner")

print("\n--- Plan for Filter then Join ---")
filtered_then_joined_df.explain(True)


# Explanation:
# When comparing the 'Optimized Logical Plan' and 'Physical Plan' for both scenarios, you'll observe that Spark's Catalyst optimizer often pushes down predicates (filters) as early as possible.
# Even if you write 'join then filter', Spark's optimizer will try to reorder operations to 'filter then join' if it's more efficient.

# However, explicitly writing 'filter then join' is often clearer and can sometimes provide hints to the optimizer.
# The 'Filter then Join' approach is generally more efficient because:
# 1. It reduces the size of the 'rides_df' before the join operation, meaning less data is shuffled and processed during the join.
# 2. Joins are expensive operations, and operating on smaller datasets minimizes this cost.
# Spark's optimizer is very intelligent and might optimize 'join then filter' to effectively become 'filter then join' under the hood (predicate pushdown).
# So, while the logical representation might differ, the physical execution plan (especially the optimized one) might end up being very similar for both, showcasing Spark's optimization capabilities.


--- Plan for Join then Filter ---
== Parsed Logical Plan ==
'Filter ('distance_km > 10)
+- Filter (status#5 = Completed)
   +- Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
      +- Join Inner, (city#2 = city#6)
         :- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
         +- LogicalRDD [city#6, surge_multiplier#7], false

== Analyzed Logical Plan ==
city: string, ride_id: string, user_id: string, distance_km: double, duration_seconds: bigint, status: string, surge_multiplier: double
Filter (distance_km#3 > cast(10 as double))
+- Filter (status#5 = Completed)
   +- Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
      +- Join Inner, (city#2 = city#6)
         :- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
         +- LogicalRDD [city#6, surge_multiplier#7], false

== 

# EXERCISE SET 3 — PARTITIONS & SHUFFLE

# Exercise 3.1

Check the number of partitions of rides_df .
Tasks:
Repartition into 4 partitions

Coalesce into 1 partition
Observe number of output files when writing to Parquet

In [11]:
import os
import shutil

# 1. Check initial number of partitions of rides_df
initial_partitions = rides_df.rdd.getNumPartitions()
print(f"Initial number of partitions for rides_df: {initial_partitions}")

# Define temporary output directories
repartitioned_output_path = "/tmp/rides_repartitioned_4"
coalesced_output_path = "/tmp/rides_coalesced_1"

# Clean up previous runs if any
if os.path.exists(repartitioned_output_path):
    shutil.rmtree(repartitioned_output_path)
if os.path.exists(coalesced_output_path):
    shutil.rmtree(coalesced_output_path)

# 2. Repartition into 4 partitions
repartitioned_rides_df = rides_df.repartition(4)
repartitioned_partitions = repartitioned_rides_df.rdd.getNumPartitions()
print(f"Number of partitions after repartitioning to 4: {repartitioned_partitions}")

# Write to Parquet and observe output files
repartitioned_rides_df.write.mode("overwrite").parquet(repartitioned_output_path)
repartitioned_files = [f for f in os.listdir(repartitioned_output_path) if f.endswith('.parquet')]
print(f"Number of output Parquet files after repartitioning: {len(repartitioned_files)}")

# 3. Coalesce into 1 partition
coalesced_rides_df = rides_df.coalesce(1)
coalesced_partitions = coalesced_rides_df.rdd.getNumPartitions()
print(f"Number of partitions after coalescing to 1: {coalesced_partitions}")

# Write to Parquet and observe output files
coalesced_rides_df.write.mode("overwrite").parquet(coalesced_output_path)
coalesced_files = [f for f in os.listdir(coalesced_output_path) if f.endswith('.parquet')]
print(f"Number of output Parquet files after coalescing: {len(coalesced_files)}")

# Clean up temporary directories
shutil.rmtree(repartitioned_output_path)
shutil.rmtree(coalesced_output_path)

Initial number of partitions for rides_df: 2
Number of partitions after repartitioning to 4: 4
Number of output Parquet files after repartitioning: 4
Number of partitions after coalescing to 1: 1
Number of output Parquet files after coalescing: 1


# Exercise 3.2

Repartition rides by city .
Tasks:
Run explain(True)
Identify whether a shuffle is introduced

In [12]:
repartitioned_rides_by_city_df = rides_df.repartition("city")
repartitioned_rides_by_city_df.explain(True)

== Parsed Logical Plan ==
'RepartitionByExpression ['city]
+- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Analyzed Logical Plan ==
ride_id: string, user_id: string, city: string, distance_km: double, duration_seconds: bigint, status: string
RepartitionByExpression [city#2]
+- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Optimized Logical Plan ==
RepartitionByExpression [city#2]
+- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange hashpartitioning(city#2, 200), REPARTITION_BY_COL, [plan_id=182]
   +- Scan ExistingRDD[ride_id#0,user_id#1,city#2,distance_km#3,duration_seconds#4L,status#5]



# EXERCISE SET 4 — JOIN WITHOUT BROADCAST (BAD DAG)

# Exercise 4.1

Join rides_df with surge_df on city without using broadcast.
Tasks:
Run explain(True)
Identify:
Join type
Exchange operators
Sort operations
Stage boundaries

In [13]:
joined_df_no_broadcast = rides_df.join(surge_df, on="city", how="inner")
joined_df_no_broadcast.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [city])
:- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
+- LogicalRDD [city#6, surge_multiplier#7], false

== Analyzed Logical Plan ==
city: string, ride_id: string, user_id: string, distance_km: double, duration_seconds: bigint, status: string, surge_multiplier: double
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
   +- LogicalRDD [city#6, surge_multiplier#7], false

== Optimized Logical Plan ==
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- Filter isnotnull(city#2)
   :  +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
   +- Filter isnotnull(city#6)
    

# Exercise 4.2

Apply a filter ( distance_km > 10 ) before the join.
Tasks:
Observe whether shuffle is removed
Explain why or why not

In [14]:
filtered_rides_df = rides_df.filter("distance_km > 10")
joined_df_filtered_before_join = filtered_rides_df.join(surge_df, on="city", how="inner")

print("\n--- Plan for Filter before Join ---")
joined_df_filtered_before_join.explain(True)

# Explanation:
# Even with the filter applied before the join, the Physical Plan still shows SortMergeJoin with Exchange hashpartitioning operations.
# This is because the small size of 'surge_df' (our lookup table) does not automatically trigger a Broadcast Hash Join by Spark's optimizer
# in this specific setup, even though it would be more efficient for small lookup tables.
# The filter on 'rides_df' only reduces the amount of data processed *before* the shuffle and sort for the join, but it does not eliminate the need for the shuffle itself
# for a SortMergeJoin, which requires both sides to be co-located and sorted by the join key.
# To explicitly remove the shuffle for the smaller DataFrame, we would need to use a broadcast hint or configure Spark to broadcast smaller tables automatically.


--- Plan for Filter before Join ---
== Parsed Logical Plan ==
'Join UsingJoin(Inner, [city])
:- Filter (distance_km#3 > cast(10 as double))
:  +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
+- LogicalRDD [city#6, surge_multiplier#7], false

== Analyzed Logical Plan ==
city: string, ride_id: string, user_id: string, distance_km: double, duration_seconds: bigint, status: string, surge_multiplier: double
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- Filter (distance_km#3 > cast(10 as double))
   :  +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
   +- LogicalRDD [city#6, surge_multiplier#7], false

== Optimized Logical Plan ==
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- Filter ((isnotnull(dista

# EXERCISE SET 5 — BROADCAST JOIN (GOOD DAG)

# Exercise 5.1

Apply a broadcast hint to surge_df .
Tasks:
Run explain(True)
Identify:
Join type
BroadcastExchange
Disappearance of shuffles

In [15]:
from pyspark.sql.functions import broadcast

joined_df_broadcast = rides_df.join(broadcast(surge_df), on="city", how="inner")

print("\n--- Plan for Broadcast Join ---")
joined_df_broadcast.explain(True)


--- Plan for Broadcast Join ---
== Parsed Logical Plan ==
'Join UsingJoin(Inner, [city])
:- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [city#6, surge_multiplier#7], false

== Analyzed Logical Plan ==
city: string, ride_id: string, user_id: string, distance_km: double, duration_seconds: bigint, status: string, surge_multiplier: double
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
   +- ResolvedHint (strategy=broadcast)
      +- LogicalRDD [city#6, surge_multiplier#7], false

== Optimized Logical Plan ==
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6), rightHint=(strategy=broadcast)
   :- Filter isnotnull

# EXERCISE SET 6 — DAG INTERPRETATION

# Exercise 6.1

From the physical plan:
Identify all expensive operators
Classify them as CPU, memory, or network heavy

In [17]:
# Common Expensive Operators and their Resource Classification:

# 1. Shuffle operations (e.g., 'Exchange HashPartitioning', 'Exchange Sort')
#    - Resource Heavy: Primarily Network and Disk I/O.
#    - Explanation: Involve moving data between executors/nodes. Data is serialized, sent over the network, deserialized, and potentially written to disk (shuffle spill). Highly impactful on performance for large datasets.

# 2. Sort operations ('Sort')
#    - Resource Heavy: Primarily CPU and Memory, potentially Disk I/O.
#    - Explanation: Require comparing and ordering data. If data fits in memory, it's CPU/memory-intensive. If not, it spills to disk, becoming Disk I/O heavy.

# 3. Joins (e.g., 'SortMergeJoin', 'ShuffleHashJoin', 'BroadcastHashJoin')
#    - Resource Heavy: Varies by type.
#        - SortMergeJoin: Heavily Network, Disk I/O (due to shuffles and sorts), CPU.
#        - ShuffleHashJoin: Heavily Network, Disk I/O (due to shuffles for both tables), Memory/CPU.
#        - BroadcastHashJoin: Primarily Memory (for broadcasting small table) and Network (for initial broadcast). Very CPU-efficient for lookups once broadcasted.
#    - Explanation: Joins often involve shuffles and comparisons. The most resource-intensive aspects typically come from the shuffling and sorting phases.

# 4. Aggregations (e.g., 'HashAggregate', 'SortAggregate')
#    - Resource Heavy: Primarily CPU and Memory, potentially Disk I/O.
#    - Explanation: Involve grouping data and performing calculations. Hash aggregations use memory for hash tables. Sort aggregations involve sorting. If intermediate data is too large, it spills to disk.

# 5. Window functions ('Window')
#    - Resource Heavy: Primarily Memory, CPU, and potentially Disk I/O.
#    - Explanation: Often require partitioning and sorting data, and holding a portion of data in memory for computations within a 'window'. Can be very memory-intensive for large windows.

# Identifying these operators in a physical plan is key to understanding and optimizing Spark application performance.

# Exercise 7.2

Trigger different actions ( count , show , write ) separately.
Tasks:
Observe whether Spark recomputes the DAG
Explain behavior

In [18]:
import os
import shutil

# The DataFrame `filtered_selected_rides_df` was defined in Exercise 2.1
# It includes multiple filters and a selection.

# --- Action 1: count() ---
print("\n--- Triggering count() ---")
row_count = filtered_selected_rides_df.count()
print(f"Row count: {row_count}")
# Explanation: Spark executed the entire DAG up to this point to compute the count.

# --- Action 2: show() ---
print("\n--- Triggering show() ---")
filtered_selected_rides_df.show()
# Explanation: Spark re-executed the entire DAG to display the results.

# --- Action 3: write() ---
output_path = "/tmp/filtered_rides_output"
# Clean up previous runs if any
if os.path.exists(output_path):
    shutil.rmtree(output_path)

print("\n--- Triggering write() ---")
filtered_selected_rides_df.write.mode("overwrite").parquet(output_path)
print(f"DataFrame written to {output_path}")
# Explanation: Spark re-executed the entire DAG again to write the data.

# Clean up temporary directory
shutil.rmtree(output_path)

# Behavior Explanation:
# Spark's operations are lazy. This means that when you define a series of transformations (like filter, select),
# Spark doesn't immediately compute the results. Instead, it builds a Directed Acyclic Graph (DAG) of operations.
# The actual computation only kicks off when an 'action' is called (like count(), show(), collect(), write()).

# Crucially, for each action, Spark, by default, will re-evaluate and re-execute the *entire* DAG from the source
# DataFrame. This is why you see the '--- Triggering X() ---' messages followed by output, and each output
# corresponds to a full re-computation of the transformations defined on 'filtered_selected_rides_df'.

# This behavior can lead to inefficiencies if you perform multiple actions on the same transformed DataFrame.
# To avoid recomputing the same transformations repeatedly, you would typically use caching (e.g., .cache() or .persist())
# on intermediate DataFrames that are going to be used in multiple subsequent actions.


--- Triggering count() ---
Row count: 4

--- Triggering show() ---
+-------+---------+-----------+---------+
|ride_id|     city|distance_km|   status|
+-------+---------+-----------+---------+
|   R001|Hyderabad|       12.5|Completed|
|   R005|Hyderabad|       20.0|Completed|
|   R006|    Delhi|       25.0|Completed|
|   R008|Bangalore|       18.0|Completed|
+-------+---------+-----------+---------+


--- Triggering write() ---
DataFrame written to /tmp/filtered_rides_output


# EXERCISE SET 8 — THINKING QUESTIONS (WRITTEN)

# 1. Why does broadcast remove shuffle from the DAG?

Broadcast joins remove shuffles from the DAG primarily because of how they handle the smaller of the two DataFrames involved in the join:

Replication, Not Redistribution: Instead of shuffling both large and small DataFrames to ensure co-located join keys, a broadcast join works by collecting the entire small DataFrame (or table) on the driver and then broadcasting (sending) it to all executor nodes.

Local Hash Map: Each executor receives a full copy of the small DataFrame and stores it in memory, typically as a hash map. This makes the data available locally on every node that will process the larger DataFrame.

Local Lookups: When the larger DataFrame is processed, each partition of the large DataFrame can perform a local lookup against the in-memory hash map of the broadcasted (smaller) DataFrame. This means there's no need to exchange data between executors to find matching keys.

Elimination of Network I/O and Disk I/O: Because the smaller table is already present on each executor, the expensive network transfer (shuffle) and potential disk I/O (if data spills during shuffle/sort) associated with redistributing the large DataFrame's data based on join keys are completely avoided. The join operation transforms from a distributed, shuffle-heavy process into a series of highly efficient local hash lookups.

In essence, by proactively moving the small table to every processing unit, Spark eliminates the need for the larger table to be reorganized (shuffled) across the network, thereby removing the shuffle stage from the DAG and significantly improving performance for joins involving a small and a large table.

# 2. Why does repartition always introduce shuffle?

repartition() always introduces a shuffle because its primary purpose is to change the number of partitions or the partitioning scheme of a DataFrame. To achieve this, Spark must redistribute the data across the cluster.

Here's why:

Distributed Nature: Data in Spark DataFrames is distributed across various nodes and executors in a cluster, stored in partitions. Each partition is processed independently.

Changing Partition Boundaries: When you call repartition(N) (to change the number of partitions to N) or repartition('column_name') (to repartition by a specific column), Spark needs to ensure that the data is correctly distributed according to the new partitioning logic.

Data Movement: To achieve the new distribution, data often needs to move from one physical machine or executor to another. For example, if a row was in partition X on machine A and, after repartitioning, needs to be in partition Y on machine B, it must be sent over the network.

Shuffle Mechanism: This movement and redistribution of data across the network (and potentially disk) is precisely what a "shuffle" operation is in Spark. Spark uses hashing or sorting mechanisms to determine which new partition each row should go into and then performs the data transfer.

coalesce() vs. repartition(): While coalesce() can sometimes avoid a full shuffle if it's only reducing the number of partitions by merging existing ones on the same machines, repartition() guarantees a full shuffle because it rebuilds the partitions from scratch according to the specified number or column, ensuring an even distribution and potentially requiring data to move between all nodes.

# 3. Why is coalesce cheaper than repartition?

`coalesce()` is generally cheaper than `repartition()` primarily because of how each operation handles data movement across the cluster.

Here's a breakdown:

1.  **Purpose**: Both `coalesce()` and `repartition()` are used to change the number of partitions in a DataFrame.

2.  **`repartition()` (Always Shuffles)**:
    *   **Goal**: To change the number of partitions to a specified amount and ensure an **even distribution** of data across these new partitions.
    *   **Mechanism**: `repartition()` *always* performs a full shuffle of the data. This means all data is read from the existing partitions, then redistributed across the cluster to the new set of partitions according to a hashing or range partitioning strategy. This ensures that the new partitions are roughly equal in size.
    *   **Cost**: Full shuffle implies significant network I/O (moving data between nodes), disk I/O (if data spills to disk during shuffle), and CPU overhead (for serialization/deserialization and hashing).

3.  **`coalesce()` (Minimizes Shuffle)**:
    *   **Goal**: To reduce the number of partitions to a specified amount, while **minimizing data movement**.
    *   **Mechanism**: `coalesce()` attempts to achieve the new, smaller number of partitions by merging existing partitions on the same physical nodes. It avoids a full shuffle by trying to combine partitions that are already co-located. If you reduce the number of partitions from `M` to `N` (where `N < M`), `coalesce()` will only move data to `N` nodes (or combine partitions within existing nodes) instead of redistributing all data across `N` new partitions.
    *   **Cost**: This approach is much cheaper because it avoids the full-scale data redistribution (shuffle) inherent in `repartition()`. It can lead to imbalanced partitions (some partitions might end up with more data than others) if the initial data distribution is highly skewed, but the trade-off is often worth it for the reduced cost.

**In summary**:

*   `repartition()` is like starting fresh with new partitions, ensuring even distribution but always incurring the cost of a full shuffle.
*   `coalesce()` is like merging existing partitions, minimizing data movement and thus being cheaper, but potentially leading to uneven partition sizes.

Therefore, if your goal is only to reduce the number of partitions and you are willing to accept potentially uneven partition sizes, `coalesce()` is the more efficient and cheaper option.

# 4. Why does Spark delay execution until an action?

Spark delays execution until an action primarily due to its lazy evaluation model. This design choice offers several significant advantages:

1.  **Optimization through DAGs (Directed Acyclic Graphs)**:
    *   When you define transformations (like `filter()`, `select()`, `join()`), Spark doesn't immediately compute the results. Instead, it builds a logical plan, represented as a DAG, of all the operations you want to perform.
    *   Only when an action (like `show()`, `count()`, `collect()`, `write()`) is triggered does Spark's Catalyst optimizer analyze this entire DAG. It can then optimize the sequence of operations, combine transformations, eliminate unnecessary steps (e.g., predicate pushdown, column pruning), and choose the most efficient physical execution plan.
    *   If Spark executed every transformation immediately, it wouldn't have the holistic view of the entire pipeline needed for these global optimizations.

2.  **Efficiency and Resource Management**:
    *   **Reduced I/O**: By knowing the full lineage of operations, Spark can minimize data reads and writes. For example, if you filter a DataFrame and then select only a few columns, Spark can optimize to read only the necessary columns from the start and apply the filter early, avoiding reading and processing data that will eventually be discarded.
    *   **Avoiding Intermediate Materialization**: Spark avoids creating unnecessary intermediate DataFrames or RDDs in memory or on disk. This saves storage space and network I/O.
    *   **Fault Tolerance**: Lazy evaluation is key to Spark's fault tolerance. If a node fails during execution, Spark can re-execute only the lost partitions of the DAG from the last checkpoint or source data, rather than recomputing the entire dataset from scratch.

3.  **Conciseness and Expressiveness**: It allows developers to write complex data transformation pipelines in a more declarative and readable way, focusing on *what* they want to achieve rather than *how* to execute it step-by-step.

In summary, lazy evaluation enables Spark to perform powerful query optimizations, manage resources more efficiently, and provide robust fault tolerance, all of which contribute to its performance and scalability for big data processing.